In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [59]:
df = pd.read_csv('climate_action_data.csv')

In [60]:
print("DataFrame Columns:")
print(df.columns)
print("\nInitial DataFrame head:")
print(df.head())
print("\nDataFrame info:")
df.info()
print("\nDescriptive statistics (initial):")
print(df.describe(include='all'))
print("\nMissing values before cleaning:")
print(df.isnull().sum())
print("\nData types before cleaning:")
print(df.dtypes)
df.isnull().sum()

DataFrame Columns:
Index(['Sensor_ID', 'Date', 'Soil_Moisture(%)', 'Soil_pH', 'Temperature(C)',
       'Humidity(%)', 'Crop_Type', 'Fertilizer_Recommended(kg/ha)',
       'Irrigation_Recommended(mm)', 'Drone_Image_ID'],
      dtype='object')

Initial DataFrame head:
  Sensor_ID        Date Soil_Moisture(%) Soil_pH Temperature(C) Humidity(%)  \
0  SEN-1000  2025-01-01            36.22    6.98           21.3        50.4   
1  SEN-1001  2025-01-02            76.55    5.03           23.5        34.3   
2  SEN-1002  2025-01-03            61.24    6.52           21.9        54.6   
3  SEN-1003  2025-01-04            51.91    6.62           24.0        48.7   
4  SEN-1004  2025-01-05            20.92    5.98           19.2        70.6   

  Crop_Type Fertilizer_Recommended(kg/ha) Irrigation_Recommended(mm)  \
0  Tomatoes                          67.6                       26.6   
1     Wheat                         130.4                        8.3   
2  Tomatoes                          36.0 

Sensor_ID                        0
Date                             4
Soil_Moisture(%)                 0
Soil_pH                          0
Temperature(C)                   0
Humidity(%)                      0
Crop_Type                        4
Fertilizer_Recommended(kg/ha)    0
Irrigation_Recommended(mm)       0
Drone_Image_ID                   0
dtype: int64

In [61]:
numeric_cols = [
    'Soil_Moisture(%)','Soil_pH','Temperature(C)','Humidity(%)','Fertilizer_Recommended(kg/ha)','Irrigation_Recommended(mm)'
]
categorical_cols = [
   'Crop_Type','Drone_Image_ID','Sensor_ID']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce') # Ensure numeric
        df[col] = df[col].fillna(df[col].median())
    else:
        print(f"Warning: Numeric column '{col}' (assumed) not found in DataFrame during cleaning.")

for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mode()[0])
    else:
        print(f"Warning: Categorical column '{col}' (assumed) not found in DataFrame during cleaning.")


In [62]:
df.isnull().sum()


Sensor_ID                        0
Date                             4
Soil_Moisture(%)                 0
Soil_pH                          0
Temperature(C)                   0
Humidity(%)                      0
Crop_Type                        0
Fertilizer_Recommended(kg/ha)    0
Irrigation_Recommended(mm)       0
Drone_Image_ID                   0
dtype: int64

In [63]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

print(f"Number of duplicates: {df.duplicated().sum()}")
df = df.drop_duplicates()
print(f"Duplicate dates: {df['Date'].duplicated().sum()}")
df = df.drop_duplicates(subset=['Date'], keep='first')

Number of duplicates: 111
Duplicate dates: 3


In [64]:
df.head()


,Sensor_ID,Date,Soil_Moisture(%),Soil_pH,Temperature(C),Humidity(%),Crop_Type,Fertilizer_Recommended(kg/ha),Irrigation_Recommended(mm),Drone_Image_ID
0,SEN-1000,2025-01-01,36.22,6.98,21.3,50.4,Tomatoes,67.6,26.6,IMG-2000
1,SEN-1001,2025-01-02,76.55,5.03,23.5,34.3,Wheat,130.4,8.3,IMG-2001
2,SEN-1002,2025-01-03,61.24,6.52,21.9,54.6,Tomatoes,36.0,29.4,IMG-2002
3,SEN-1003,2025-01-04,51.91,6.62,24.0,48.7,Maize,85.8,19.6,IMG-2003
4,SEN-1004,2025-01-05,20.92,5.98,19.2,70.6,Wheat,75.6,29.9,IMG-2004


In [ ]:
numeric_df_for_corr = df.select_dtypes(include=np.number) 
if not numeric_df_for_corr.empty and len(numeric_df_for_corr.columns) > 1:
    corr_matrix = numeric_df_for_corr.corr()
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
    plt.title('Correlation Matrix of Numeric Features')
    plt.tight_layout()
    plt.savefig('correlation_matrix.png')
    plt.show()
    print("\nCorrelation matrix plot generated as 'correlation_matrix.png'.")

else:
    print("\nNot enough numeric columns available for a meaningful correlation matrix after cleaning.")

# Example Analyses (adjust column names based on actual data)
# These will only run if the columns exist.
primary_numeric_metric = None
if 'Emissions Reduction (tCO2e/yr)' in numeric_cols:
    primary_numeric_metric = 'Emissions Reduction (tCO2e/yr)'
elif numeric_cols: # Fallback to the first available numeric column
    primary_numeric_metric = numeric_cols[0]

primary_categorical_attribute = None
if 'Sector' in categorical_cols:
    primary_categorical_attribute = 'Sector'
elif categorical_cols: # Fallback
    primary_categorical_attribute = categorical_cols[0]
    
primary_date_column = None
if date_cols:
    primary_date_column = date_cols[0]
elif 'Date' in df.columns:
    primary_date_column = 'Date'


if primary_categorical_attribute and primary_numeric_metric:
    avg_metric_by_category = df.groupby(primary_categorical_attribute)[primary_numeric_metric].mean().sort_values(ascending=False)
    print(f"\nAverage {primary_numeric_metric} by {primary_categorical_attribute}:")
    print(avg_metric_by_category)

if 'Region' in categorical_cols and 'Investment (USD)' in numeric_cols:
    total_investment_by_region = df.groupby('Region')['Investment (USD)'].sum().sort_values(ascending=False)
    print("\nTotal Investment (USD) by Region:")
    print(total_investment_by_region)

# 5. Visualize Data

if primary_date_column and primary_numeric_metric:
    # Ensure the date column is datetime and sort
    df[primary_date_column] = pd.to_datetime(df[primary_date_column])
    df_sorted_for_ts = df.sort_values(by=primary_date_column)
    
    plt.figure(figsize=(14, 7))
    # If there are many data points, resampling might be good, e.g., df_sorted_for_ts.groupby(pd.Grouper(key=primary_date_column, freq='M'))[primary_numeric_metric].sum().plot()
    # For now, a simple plot of sums per date (if multiple entries per date) or direct plot if dates are unique enough.
    df_sorted_for_ts.groupby(primary_date_column)[primary_numeric_metric].sum().plot()
    plt.title(f'Total {primary_numeric_metric} Over Time ({primary_date_column})')
    plt.xlabel(primary_date_column)
    plt.ylabel(f'Total {primary_numeric_metric}')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('time_series_plot.png')
    plt.show()
    print(f"\nTime series plot generated as 'time_series_plot.png'.")


if primary_categorical_attribute and primary_numeric_metric and not avg_metric_by_category.empty:
    plt.figure(figsize=(12, 7))
    avg_metric_by_category.plot(kind='bar', color=sns.color_palette('viridis', len(avg_metric_by_category)))
    plt.title(f'Average {primary_numeric_metric} by {primary_categorical_attribute}')
    plt.xlabel(primary_categorical_attribute)
    plt.ylabel(f'Average {primary_numeric_metric}')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('average_metric_by_category.png')
    plt.show()
    print(f"\nBar chart 'Average {primary_numeric_metric} by {primary_categorical_attribute}' generated as 'average_metric_by_category.png'.")


if 'Region' in categorical_cols and 'Investment (USD)' in numeric_cols and (not total_investment_by_region.empty if 'total_investment_by_region' in locals() else False):
    plt.figure(figsize=(12, 7))
    total_investment_by_region.plot(kind='bar', color=sns.color_palette('magma', len(total_investment_by_region)))
    plt.title('Total Investment (USD) by Region')
    plt.xlabel('Region')
    plt.ylabel('Total Investment (USD)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('investment_by_region.png')
    plt.show()
    print("\nBar chart 'Total Investment by Region' generated as 'investment_by_region.png'.")


# Scatter plot: Example - Investment vs. Emissions Reduction
if 'Investment (USD)' in numeric_cols and 'Emissions Reduction (tCO2e/yr)' in numeric_cols:
    plt.figure(figsize=(10, 6))
    hue_attribute = primary_categorical_attribute if primary_categorical_attribute else None # Use a categorical column for hue if available
    sns.scatterplot(data=df, x='Investment (USD)', y='Emissions Reduction (tCO2e/yr)', hue=hue_attribute, s=100, alpha=0.7)
    plt.title('Investment vs. Emissions Reduction')
    plt.xlabel('Investment (USD)')
    plt.ylabel('Emissions Reduction (tCO2e/yr)')
    plt.grid(True)
    if hue_attribute:
        plt.legend(title=hue_attribute, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig('investment_vs_reduction_scatter.png')
    plt.show()
    print("\nScatter plot 'Investment vs. Emissions Reduction' generated as 'investment_vs_reduction_scatter.png'.")


if primary_numeric_metric:
    plt.figure(figsize=(10, 6))
    sns.histplot(df[primary_numeric_metric].dropna(), kde=True, bins=30)
    plt.title(f'Distribution of {primary_numeric_metric}')
    plt.xlabel(primary_numeric_metric)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('metric_distribution.png')
    plt.show()
    print(f"\nHistogram 'Distribution of {primary_numeric_metric}' generated as 'metric_distribution.png'.")
